## Bring in the Data from APSIM and prepare for graphing

In [120]:
import sqlite3
import pandas as pd
from mpl_toolkits.mplot3d import axes3d
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
from datetime import datetime
from datetime import datetime, timedelta

con = sqlite3.connect('C:\ApsimX\Prototypes\MultiPoreWater/ReportDetail.db')
APSIMData = pd.read_sql("Select * from Report,Simulations where Report.SimulationID = Simulations.ID",con, index_col = ['TimeStep'])
APSIMData = APSIMData.drop(labels = ['SimulationID', 'ZoneName','ID'],axis=1)
PoreWater = APSIMData.ix[:,[X for X in APSIMData.columns if 'PW' in X]]
ProfileData = pd.read_sql("Select * from Initial,Simulations where Initial.SimulationID = Simulations.ID",con)
Thickness = ProfileData.ix[:,[X for X in ProfileData.columns if 'Thick' in X]]
Timing = APSIMData.ix[:,0:3]
Timing.ix[:,'DateTime'] = [datetime.strptime(Timing.ix[X,0], '%Y-%m-%d %H:%M:%S') for X in Timing.index]
Timing.ix[:,'Date0'] = [Timing.ix[X,'DateTime'] - timedelta(hours=12) for X in Timing.index]
Timing.ix[:,'DateTime'] = [Timing.ix[X,'Date0'] + timedelta(hours=int(Timing.ix[X,'Hour'])) for X in Timing.index]
Timing.ix[:,'Now'] = [datetime.strftime(Timing.ix[X,'DateTime'],'%H:%M:%S %d-%b-%Y') for X in Timing.index]

Depth = []
depth = 0
for l in Thickness.columns:
    Depth.append(depth)
    depth += Thickness.ix[:,l][0]

LayerNum = 7
PoreComNum = 7

LayerIndices = []
for i in range(1,LayerNum +1):
    LayerIndices.append('L'+str(i))
    
CompartmentIndices = []
for i in range(1,PoreComNum +1):
    CompartmentIndices.append('C'+str(i))    

PoreColumnIndex = pd.MultiIndex.from_product([LayerIndices,CompartmentIndices],names=['Layer','PoreComponent'])
PoreWater.columns = PoreColumnIndex

def WetnessData(i):
    Wetness = []
    for l in range(1,LayerNum+1):
        Wetness.append(list(PoreWater.ix[i+1,'L'+str(l)].values))
    return np.array(Wetness)

## Make an animated heat map of the water content in the soil

In [129]:
fig = plt.figure(figsize=(10,12))
ax1=plt.subplot2grid((6,1),(0,0),rowspan=1,xticks=[],yticks=[])
ax2=plt.subplot2grid((6,1),(1,0),rowspan=5,
                     ylabel='Depth (mm)',xlabel='Large --> Small pores')
ax2.invert_yaxis()
ax2.tick_params(labelsize=15)
ax2.xaxis.label.set_fontsize(30)
ax2.yaxis.label.set_fontsize(30)

pores = range(1,PoreComNum+1)
layers = Depth
Pores, Layers = np.meshgrid(pores, layers)

ims = []
for i in range(150):
    plt1 = ax1.text(0.1,0.5,Timing.ix[i+1,'Now'],fontsize=40)
    plt2 = ax2.pcolor(pores, layers, WetnessData(i), cmap=cm.jet_r, norm=plt.Normalize(0, 1))
    ims.append([plt1,plt2])
im_ani = animation.ArtistAnimation(fig, ims, interval=20, repeat_delay=1000,
                                   blit=True)
plt.show()

## Graphs below here work, just not very well

In [222]:
fig = plt.figure()
ax = axes3d.Axes3D(fig)

pores = range(1,PoreComNum+1)
layers = Depth
Pores, Layers = np.meshgrid(pores, layers)
Wetness = WetnessData(1)
wframe = ax.plot_wireframe(Layers,Pores, Wetness)
ax.set_zlim(0,1)

def update(i, ax, fig):
    ax.cla()
    Wetness = WetnessData(i)
    wframe = ax.plot_wireframe(Layers, Pores, Wetness)
    ax.set_zlim(0,1)
    ax.set_xlabel('Profile Depth (mm)')
    ax.set_ylabel('L pores --> S pores')
    ax.set_zlabel('Relative Water Content')
    return wframe,

ani = animation.FuncAnimation(fig, update, 
       frames=xrange(PoreWater.index.size), 
       fargs=(ax, fig), interval=0.001)
plt.show()

In [8]:
fig = plt.figure()
ax = fig.gca(projection='3d')

def MakeFrame(i, ax, fig):
    ax.cla()
    Wetness = WetnessData(i)
    surface = ax.plot_surface(Layers, Pores, Wetness, cstride=1, rstride=1, cmap=cm.jet, antialiased=False)
    ax.set_zlim(0,1)
    ax.set_xlabel('Profile Depth (mm)')
    ax.set_ylabel('L pores --> S pores')
    ax.set_zlabel('Relative Water Content')
    ax.text(0,0,1.5,Timing.ix[i+1,'Now'])
    return surface,

pores = range(1,PoreComNum+1)
layers = Depth
Pores, Layers = np.meshgrid(pores, layers)

MakeFrame(1,ax,fig)

ani = animation.FuncAnimation(fig, MakeFrame, 
        frames=xrange(PoreWater.index.size), 
        fargs=(ax, fig), interval=1)
plt.show()

In [19]:
fig = plt.figure()
ax = fig.gca(projection='3d')

pores = range(1,PoreComNum+1)
layers = Depth
Pores, Layers = np.meshgrid(pores, layers)

def MakeFrame(i, ax, fig):
    Wetness = WetnessData(i)
    surface = ax.plot_surface(Layers, Pores, Wetness, cstride=1, rstride=1, cmap=cm.jet, antialiased=False)
    ax.set_zlim(0,1)
    ax.set_xlabel('Profile Depth (mm)')
    ax.set_ylabel('L pores --> S pores')
    ax.set_zlabel('Relative Water Content')
    ax.text(0,0,1.5,Timing.ix[i+1,'Now'])
    return surface,

Frames = []
for i in range(1000):
    Frames.append(MakeFrame(i,ax,fig))

In [21]:
ani = animation.ArtistAnimation(fig, Frames, interval=100)
plt.show()

In [33]:
##Example heat map
fig2 = plt.figure()

x = np.arange(-9, 10)
y = np.arange(-9, 10).reshape(-1, 1)
base = np.hypot(x, y)
ims = []
for add in np.arange(15):
    ims.append((plt.pcolor(x, y, base + add, norm=plt.Normalize(0, 30)),))

im_ani = animation.ArtistAnimation(fig2, ims, interval=50, repeat_delay=3000,
                                   blit=True)
plt.show()